In [60]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm 
import glob
import re
import requests
import io
from Bio.Blast import NCBIWWW

In [61]:
kif_duke = pd.read_csv("../../data/kif/kif_duke.csv")
kif_duke.head()

,Species/protein,Molecular mass (kDa),Motor polarity & velocity,Subcellular localization,Comments,Seq,Type
0,H. sapiens KHC,109,ND,ND,NaN,HsxKHC AECSIKVMCRFRPLNEAEILRGDKFIPKFK---GDE...,1
1,H. sapiens neuronal KHC,117,ND,Cell body in cultured neurons,Specific to neural tissue,HsnKHC NECSIKVLCRFRPLNQAEILRGDKFIPIFQ---GDD...,1
2,H. sapiens KHC,110,ND,Uniform in cultured neurons,Ubiquitously expressed,HsKHC AECNIKVMCRFRPLNESEVNRGDKYIAKFQ---GED...,1
3,D. melanogaster KHC,110,"Plus, 54 µm/min",Diffuse in cytoplasm,Required for viability & neuromuscular function,DmKHC AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEEN...,1
4,C. elegans KHC (Unc-116),92,ND,ND,"Axonal transport of synaptic vesicles, also or...",CeKHC AECGVQVFCRIRPLNKTEEKNADRFLPKFP---SED...,1


In [62]:
def get_uniprot_entry(acc):
    url = "https://www.uniprot.org/uniprot/?query="+acc+"&sort=score&columns=id,entry name,reviewed,protein names,genes,organism,organism-id,lineage(all),sequence,length&format=tab"
    try:
        urlData = requests.get(url).content
        rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')),sep = '\t')
    except:
        rawData = pd.DataFrame()
    return rawData 

In [63]:
kif_duke.loc[(~pd.isnull(kif_duke["Motor polarity & velocity"])) & (kif_duke["Motor polarity & velocity"]!="ND"),:]

,Species/protein,Molecular mass (kDa),Motor polarity & velocity,Subcellular localization,Comments,Seq,Type
3,D. melanogaster KHC,110,"Plus, 54 µm/min",Diffuse in cytoplasm,Required for viability & neuromuscular function,DmKHC AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEEN...,1
6,L. pealii KHC,109,"Plus, 30 µm/min",Membranous vesicles,NaN,LpKHC SECNIKVICRVRPLNEAEERAGSKFILKFP---TDD...,1
10,N. crassa KHC (NKin),103,"Plus, 120-180 µm/min",ND,High velocity of microtubule transport; no cop...,NcKHC SANSIKVVARFRPQNRVEIESGGQPIVTFQ---GPD...,1
22,D. melanogaster Klp68D,88,"Plus, 18 µm/min",ND,NaN,Dmklp68D PNECVQVVVRCRPMSNRERSERSPEVVNVYPNRGVV...,2
25,M. musculus KIF3A,80,"Plus, 36 µm/min",Microsomes/ synaptic vesicles,NaN,MmKIF3A SCDNVKVVVRCRPLNEREKSMCYRQAVSVDEMRGTI...,2
26,M. musculus KIF3B,85,"Plus, 18 µm/min","Neurons, cell bodies, axons and dendrites",NaN,MmKIF3B SSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQV...,2
29,S. purpuratus KRP85,79,"Plus, 24 µm/min",ND,NaN,SpKRP85 GNDNVRVVVRCRPLNSKETGQGFKSVVKMDEMRGTV...,2
30,S. purpuratus KRP95,84,"Plus, 24 µm/min",ND,NaN,SpKRP95 SAETVKVVVRCRPMNSKEISQGHKRIVEMDNKRGLV...,2
44,M. musculus KIF1A,192,"Plus, 72 µm/min",Enriched in axons,"Associated with synaptic vesicles, monomeric?",MmKIF1A AGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTI...,3
45,M. musculus KIF1B,130,"Plus, 40 µm/min",Co-localizes with mitochondria,"Transports mitochondria, monomeric?",MmKIF1B SGASVKVAVRVRPFNSRETSKESKCIIQMQGNSTSI...,3


In [64]:
kif_duke.shape

(182, 7)

In [65]:
kif_list_jp = pd.read_csv("../../data/kif/KIFlist.txt", sep = "\t")
kif_list_jp.head()



,Family,Subfamily,KIF,Alias
0,Kinesin-1,KIF5(KHC),HsKIF5A,HsnKHC
1,Kinesin-1,KIF5(KHC),MmKIF5A,NaN
2,Kinesin-1,KIF5(KHC),RnKIF5A,NaN
3,Kinesin-1,KIF5(KHC),DrKIF5A,NaN
4,Kinesin-1,KIF5(KHC),MsFKIF5,NaN


In [66]:
kif_list_jp.groupby("Family").count()

,Subfamily,KIF,Alias
Family,,,
Kinesin-1,42,42,3
Kinesin-10,16,16,1
Kinesin-11,25,25,0
Kinesin-12,25,25,1
Kinesin-13,43,43,1
Kinesin-14A,44,44,1
Kinesin-14B,63,63,1
Kinesin-2,43,43,0
Kinesin-3,88,88,4


In [67]:
kif_list_jp.shape

(626, 4)

In [68]:
kif_list_jp_acc = pd.read_csv("../../data/kif/AccNo.txt", sep = "\t",header = None)
kif_list_jp_acc.head()

,0,1,2
0,AgKHC,XP_310522,ENSANGG00000014820
1,AgKlp31E,XP_317685,ENSANGP00000002307
2,AgNcd,XP_307936,ENSANGP00000006252
3,AgKin73,XP_308280,ENSANGP00000009361
4,AgKlp68D,XP_311552,ENSANGP00000010166


In [69]:
kif_list_jp_acc.columns = ["KIF","prot_acc","ensembl"]

In [70]:
kif_list_jp_acc.shape

(626, 3)

In [71]:
# merge the accession table and the information table of japan database
kif_jp = kif_list_jp_acc.merge(kif_list_jp,on = "KIF")

In [72]:
kif_duke["KIF"] = kif_duke["Seq"].apply(lambda s: str(s).split(" ")[0])

In [73]:
kif_jp.merge(kif_duke,on = "KIF")

,KIF,prot_acc,ensembl,Family,Subfamily,Alias,Species/protein,Molecular mass (kDa),Motor polarity & velocity,Subcellular localization,Comments,Seq,Type
0,AtKCBP,AAC49901,NaN,Kinesin-14B,KCBP,NaN,H. sapiens KIFC2,NaN,NaN,NaN,NaN,AtKCBP MKGKIRVYCRIRPLNEKESSEREKQMLTTVDEFTVE...,14
1,AtT5A14.3,AAD10640,NaN,Kinesin-14B,TBK5,NaN,A. thaliana T5A14.3,100,NaN,NaN,NaN,AtT5A14.3 LKGNIRVFCRVKPLGATEKLRPPVASDTRNVIIKL-...,14
2,CfDSK1,Q39493,NaN,Kinesin-13,Others,NaN,C. fusiformis DSK1,63,ND,Spindle,NaN,CfDSK1 TNSNICIAVRKRPISDKERQKLDHDSVSCFQNKVWI...,13
3,CgCHO1,CAA58558,NaN,Kinesin-6,KIF23/CHO1,NaN,C. griseus CHO1,109,ND,"Spindle, midbody",Separates poles at anaphase B?,CgCHO1 LKDPVGVYCRVRPLSCPDQECCVEVVSNTTVQLHTP...,6
4,CgCHO2,CAA58559,NaN,Kinesin-14A,KIFC1,NaN,C. griseus CHO2,69,"Minus, 1-8 µm/min",Spindle/centrosomes,"Spindle at mitosis, centrosome in interphase",CgCHO2 LKGNIRVFCRVRPVLAGEPTPSPGFLLFPHGPAGPS...,14
5,CgMCAK,P70096,NaN,Kinesin-13,KIF2,NaN,C. griseus MCAK,83,ND,Kinetochore,"Localizes to kinetochore, anchors kinetochore ...",CgMCAK EEHRICVCVRKRPLNKQELAKKEIDVISVPSKCLLF...,13
6,CrFLA10,P46869,NaN,Kinesin-2,KIF3B/C,NaN,C. reinhardtii FLA10,87,ND,Flagella,NaN,CrFLA10 GSESVKVVVRCRPLNGKEKADGRSRIVDMDVDAGQV...,2
7,DmKlp10A,NP_727494,NaN,Kinesin-13,KIF2,NaN,P. falciparum L2165w,NaN,NaN,NaN,NaN,DmKlp10A DDHQITVCVRKRPISRKEVNRKEIDVISVPRKDMLI...,13
8,DmKlp59C,NP_611759,NaN,Kinesin-13,KIF2,NaN,D. melanogaster Klp59D,NaN,NaN,NaN,NaN,DmKlp59C NCHQIMVCVRKRPLRRKELADREQDVVSIPSKHTLV...,13
9,DmKlp31E,NP_609398,NaN,Kinesin-4,KIF21,NaN,D. melanogaster Klp31E,NaN,ND,NaN,NaN,DmKlp31E KDSSVRVAVRIRPQNSRELIDMCRICTTVTLGEPQI...,4


In [74]:
kif_jp.loc[kif_jp["KIF"] == "MmKIF2",:]

,KIF,prot_acc,ensembl,Family,Subfamily,Alias


In [75]:
kif_jp["prot_acc"]

0      XP_310522
1      XP_317685
2      XP_307936
3      XP_308280
4      XP_311552
         ...    
621     AAK91817
622     AAK91818
623     AAK91820
624     AAK91822
625     AAK91823
Name: prot_acc, Length: 626, dtype: object

In [76]:
get_uniprot_entry("AAK91823")

,Entry,Entry name,Status,Protein names,Gene names,Organism,Organism ID,Taxonomic lineage (all),Sequence,Length
0,Q93XF2,Q93XF2_MAIZE,unreviewed,Kinesin heavy chain (Fragment),KIN16,Zea mays (Maize),4577,"cellular organisms, Eukaryota, Viridiplantae, ...",TRPYGQTGSGKTYTMSGPGTSKEDWGVNYRALNDLFDISLSRRNAF...,405


In [77]:
kif_jp_uniprot = get_uniprot_entry(kif_jp["prot_acc"].iloc[0])
for i in range(1,kif_jp.shape[0]):
    curr = get_uniprot_entry(kif_jp["prot_acc"].iloc[i])
    if not (curr.empty):
        kif_jp_uniprot = pd.concat([kif_jp_uniprot,curr], ignore_index=True)
    if i%100 ==0:
        print(kif_jp_uniprot.shape)

(94, 10)
(170, 10)
(200, 10)
(280, 10)
(371, 10)
(433, 10)


In [102]:
kif_duke.head()

,Species/protein,Molecular mass (kDa),Motor polarity & velocity,Subcellular localization,Comments,Seq,Type,KIF,aa_seq
0,H. sapiens KHC,109,ND,ND,NaN,HsxKHC AECSIKVMCRFRPLNEAEILRGDKFIPKFK---GDE...,1,HsxKHC,AECSIKVMCRFRPLNEAEILRGDKFIPKFK---GDETVVIG-----...
1,H. sapiens neuronal KHC,117,ND,Cell body in cultured neurons,Specific to neural tissue,HsnKHC NECSIKVLCRFRPLNQAEILRGDKFIPIFQ---GDD...,1,HsnKHC,NECSIKVLCRFRPLNQAEILRGDKFIPIFQ---GDDSVVIG-----...
2,H. sapiens KHC,110,ND,Uniform in cultured neurons,Ubiquitously expressed,HsKHC AECNIKVMCRFRPLNESEVNRGDKYIAKFQ---GED...,1,HsKHC,AECNIKVMCRFRPLNESEVNRGDKYIAKFQ---GEDTVVIA-----...
3,D. melanogaster KHC,110,"Plus, 54 µm/min",Diffuse in cytoplasm,Required for viability & neuromuscular function,DmKHC AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEEN...,1,DmKHC,AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEENCISIA-----...
4,C. elegans KHC (Unc-116),92,ND,ND,"Axonal transport of synaptic vesicles, also or...",CeKHC AECGVQVFCRIRPLNKTEEKNADRFLPKFP---SED...,1,CeKHC,AECGVQVFCRIRPLNKTEEKNADRFLPKFP---SEDSISLG-----...


In [79]:
def find_aa(s):
    pattern = r'.*[\s]([\S]+)[\s].*'
    title = "ND"
    if match := re.search(pattern, s, re.IGNORECASE):
        title = match.group(1)
    return title

kif_duke["aa_seq"] = kif_duke["Seq"].apply(lambda s: find_aa(str(s)))
kif_duke["aa_seq"]

0      AECSIKVMCRFRPLNEAEILRGDKFIPKFK---GDETVVIG-----...
1      NECSIKVLCRFRPLNQAEILRGDKFIPIFQ---GDDSVVIG-----...
2      AECNIKVMCRFRPLNESEVNRGDKYIAKFQ---GEDTVVIA-----...
3      AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEENCISIA-----...
4      AECGVQVFCRIRPLNKTEEKNADRFLPKFP---SEDSISLG-----...
                             ...                        
177    DTYTGSITVTIRPKPRSVGTSRDHVGLKSPRYSQPRSNSHHGSNTF...
178    EPCHIEVILRAIPEKGLQNNESTFKIDPYENTVLFRTNNP------...
179    VGSGIITSIRIRPIGKNQGVWSHGKLSNDPYGREYIRQQT------...
180    PTEHHPAIAKRTTSSERAGAGASIAAAPSSHDLDHEDPTS------...
181    EEDAIKVFVRIRPPVEGTLTGVDGEQGLCLTALSSTTIRL------...
Name: aa_seq, Length: 182, dtype: object

In [80]:
sum(kif_duke["aa_seq"]=="ND")

54

In [81]:
kif_duke["aa_seq"][0].replace("-","")

'AECSIKVMCRFRPLNEAEILRGDKFIPKFKGDETVVIGQGKPYVFDRVLPPNTTQEQVYNACAKQIVKDVLEGYNGTIFAYGQTSSGKTHTMEGKLHDPQLMGIIPRIAHDIFDHIYSMDENLEFHIKVSYFEIYLDKIRDLLDVSKTNLAVHEDKNRVPYVKGCTERFVSSPEEVMDVIDEGKANRHVAVTNMNEHSSRSHSIFLINIKQENVETEKKLSGKLYLVDLAGSEKVSKTGAEGAVLDEAKNINKSLSALGNVISALAEGTKTHVPYRDSKMTRILQDSLGGNCRTTIVICCSPSVFNEAETKSTLMFGQRAKTIKNTVSVN'

In [82]:
fasta_string = kif_duke["aa_seq"][0].replace("-","")
result_handle = NCBIWWW.qblast("blastp", "swissprot", fasta_string)

In [83]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

In [84]:
blast_record = next(blast_records)

In [85]:
blast_record.database_length

179746915

In [100]:
get_uniprot_entry("P33175")

,Entry,Entry name,Status,Protein names,Gene names,Organism,Organism ID,Taxonomic lineage (all),Sequence,Length
0,P33175,KIF5A_MOUSE,reviewed,Kinesin heavy chain isoform 5A (Kinesin heavy ...,Kif5a Kiaa4086 Kif5 Nkhc1,Mus musculus (Mouse),10090,"cellular organisms, Eukaryota, Opisthokonta, M...",MAETNNECSIKVLCRFRPLNQAEILRGDKFIPIFQGDDSVIIGGKP...,1027
1,O08788,DCTN1_MOUSE,reviewed,Dynactin subunit 1 (150 kDa dynein-associated ...,Dctn1,Mus musculus (Mouse),10090,"cellular organisms, Eukaryota, Opisthokonta, M...",MAQSRRHMSSRTPSGSRMSTEASARPLRVGSRVEVIGKGHRGTVAY...,1281
2,Q96NW4,ANR27_HUMAN,reviewed,Ankyrin repeat domain-containing protein 27 (V...,ANKRD27 PP12899,Homo sapiens (Human),9606,"cellular organisms, Eukaryota, Opisthokonta, M...",MALYDEDLLKNPFYLALQKCRPDLCSKVAQIHGIVLVPCKGSLSSS...,1050
3,O70585,DTNB_MOUSE,reviewed,Dystrobrevin beta (DTN-B) (mDTN-B) (Beta-dystr...,Dtnb,Mus musculus (Mouse),10090,"cellular organisms, Eukaryota, Opisthokonta, M...",MIEEGGNKRKTMAEKRQLFIEMRAQNFDVIRLSTYRTACKLRFVQK...,659


In [110]:
#blast out more sequences related to kenisins with documented functions
kif_doc = kif_duke.loc[(~pd.isnull(kif_duke["Motor polarity & velocity"])) & (kif_duke["Motor polarity & velocity"]!="ND"),:]
kif_doc = kif_doc.loc[(~pd.isnull(kif_duke["Seq"])) & (kif_duke["Seq"]!="ND"),:]
kif_doc.shape

(20, 9)

In [121]:
kif_doc

,Species/protein,Molecular mass (kDa),Motor polarity & velocity,Subcellular localization,Comments,Seq,Type,KIF,aa_seq
3,D. melanogaster KHC,110,"Plus, 54 µm/min",Diffuse in cytoplasm,Required for viability & neuromuscular function,DmKHC AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEEN...,1,DmKHC,AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEENCISIA-----...
6,L. pealii KHC,109,"Plus, 30 µm/min",Membranous vesicles,NaN,LpKHC SECNIKVICRVRPLNEAEERAGSKFILKFP---TDD...,1,LpKHC,SECNIKVICRVRPLNEAEERAGSKFILKFP---TDDSISIA-----...
10,N. crassa KHC (NKin),103,"Plus, 120-180 µm/min",ND,High velocity of microtubule transport; no cop...,NcKHC SANSIKVVARFRPQNRVEIESGGQPIVTFQ---GPD...,1,NcKHC,SANSIKVVARFRPQNRVEIESGGQPIVTFQ---GPDTCTVD---SK...
22,D. melanogaster Klp68D,88,"Plus, 18 µm/min",ND,NaN,Dmklp68D PNECVQVVVRCRPMSNRERSERSPEVVNVYPNRGVV...,2,Dmklp68D,PNECVQVVVRCRPMSNRERSERSPEVVNVYPNRGVVELQNVV----...
25,M. musculus KIF3A,80,"Plus, 36 µm/min",Microsomes/ synaptic vesicles,NaN,MmKIF3A SCDNVKVVVRCRPLNEREKSMCYRQAVSVDEMRGTI...,2,MmKIF3A,SCDNVKVVVRCRPLNEREKSMCYRQAVSVDEMRGTITVHKTD----...
26,M. musculus KIF3B,85,"Plus, 18 µm/min","Neurons, cell bodies, axons and dendrites",NaN,MmKIF3B SSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQV...,2,MmKIF3B,SSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQVSVKNPK----...
29,S. purpuratus KRP85,79,"Plus, 24 µm/min",ND,NaN,SpKRP85 GNDNVRVVVRCRPLNSKETGQGFKSVVKMDEMRGTV...,2,SpKRP85,GNDNVRVVVRCRPLNSKETGQGFKSVVKMDEMRGTVQVTNPN----...
30,S. purpuratus KRP95,84,"Plus, 24 µm/min",ND,NaN,SpKRP95 SAETVKVVVRCRPMNSKEISQGHKRIVEMDNKRGLV...,2,SpKRP95,SAETVKVVVRCRPMNSKEISQGHKRIVEMDNKRGLVEVTNPK----...
44,M. musculus KIF1A,192,"Plus, 72 µm/min",Enriched in axons,"Associated with synaptic vesicles, monomeric?",MmKIF1A AGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTI...,3,MmKIF1A,AGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPK-----Q...
45,M. musculus KIF1B,130,"Plus, 40 µm/min",Co-localizes with mitochondria,"Transports mitochondria, monomeric?",MmKIF1B SGASVKVAVRVRPFNSRETSKESKCIIQMQGNSTSI...,3,MmKIF1B,SGASVKVAVRVRPFNSRETSKESKCIIQMQGNSTSIINPK-----N...


In [ ]:
E_VALUE_THRESH = 0.0001
all_kif_accs = []
for i in range(kif_doc.shape[0]):
    kif = kif_doc["KIF"].iloc[i]
    print(kif)
    fasta_string = kif_doc["aa_seq"].iloc[i].replace("-","")
    result_handle = NCBIWWW.qblast("blastp", "swissprot", fasta_string)
    blast_records = NCBIXML.parse(result_handle)
    blast_record = next(blast_records)
    print("record fetched")
    accs = ""
    for alignment in blast_record.alignments:
         for hsp in alignment.hsps:
                if hsp.expect < E_VALUE_THRESH:
                    accs = accs + alignment.accession + ";"
    print(accs)
    all_kif_accs.append((kif,accs))
#     break
    #              print("length:", alignment.length)
    #              print("e value:", hsp.expect)
    #              print(hsp.query[0:75] + "...")
    #              print(hsp.match[0:75] + "...")
    #              print(hsp.sbjct[0:75] + "...")


DmKHC
record fetched
P17210;P21613;P35978;P28738;O60282;Q61768;Q2PQA9;P33176;P34540;Q12840;Q5R9K7;P33175;Q6QLM7;P56536;O43093;Q86Z98;P48467;Q54UC9;Q86ZC1;Q9US60;Q8T135;F9W301;Q61771;O15066;P46872;Q8GW44;Q54TL0;P46871;Q965T6;Q9Y496;Q4R628;Q5R4H3;P28741;P46873;Q29DY1;P46867;Q9P2E2;Q99PW8;Q2R2P7;P46869;F4J2K4;Q9S7P3;B9F7C8;Q86VH2;F4IIS5;P82266;B9G3M6;Q54NP8;Q91783;P33174;
LpKHC
record fetched
P21613;P17210;O60282;P28738;P33175;Q12840;P35978;Q5R9K7;Q6QLM7;P33176;Q61768;Q2PQA9;P34540;P56536;O43093;P48467;Q86ZC1;Q86Z98;Q54UC9;Q9US60;Q8T135;F9W301;Q54TL0;Q8GW44;P46872;Q5R4H3;P28741;Q9Y496;Q4R628;Q61771;Q965T6;P46871;O15066;P46873;Q9P2E2;P46869;Q29DY1;Q99PW8;P46867;B9FUF9;Q9S7P3;Q2R2P7;P33174;Q91784;Q86VH2;Q2VIQ3;Q9LDN0;F4J2K4;Q27IK6;F4IIS5;
NcKHC
record fetched
P48467;Q86Z98;Q86ZC1;O43093;Q9US60;Q54UC9;P33176;Q61768;Q2PQA9;P33175;Q12840;P21613;O60282;P28738;Q5R9K7;Q6QLM7;P17210;P35978;P34540;Q61771;O15066;P46871;Q8T135;Q8GW44;Q54TL0;F9W301;Q9Y496;Q4R628;Q5R4H3;P46872;P28741;P46869;O35066;P468

In [127]:
all_kif_accs = pd.DataFrame(all_kif_accs)

In [129]:
all_kif_accs.columns = ["kif_acc","uniprot_acc"]

In [130]:
all_kif_accs

,kif_acc,uniprot_acc
0,DmKHC,P17210;P21613;P35978;P28738;O60282;Q61768;Q2PQ...
1,LpKHC,P21613;P17210;O60282;P28738;P33175;Q12840;P359...
2,NcKHC,P48467;Q86Z98;Q86ZC1;O43093;Q9US60;Q54UC9;P331...
3,Dmklp68D,
4,MmKIF3A,P28741;Q4R628;Q9Y496;Q5R4H3;P46872;P46871;Q617...
5,MmKIF3B,Q61771;O15066;P46871;Q5R706;O35066;O55165;O147...
6,SpKRP85,P46872;Q4R628;Q9Y496;Q5R4H3;P28741;P46871;Q617...
7,SpKRP95,P46871;O15066;Q61771;P46872;Q5R706;O14782;O350...
8,MmKIF1A,P33173;F1M4A4;Q12756;O60333;Q60575;O88658;O438...
9,MmKIF1B,Q60575;O88658;O60333;Q12756;F1M4A4;P33173;O350...


In [131]:
acc_str = ""
for i in range(all_kif_accs.shape[0]):
    acc_str += all_kif_accs["uniprot_acc"].iloc[i]

In [136]:
acc_ls = acc_str.split(";")[:-1]

In [137]:
len(acc_ls)

966

In [139]:
len(set(acc_ls))

239

In [143]:
len(set(kif_jp_uniprot["Entry"]).intersection(set(acc_ls)))

154

In [144]:
len (set(acc_ls) - set(kif_jp_uniprot["Entry"]))

85

In [146]:
kif_doc = kif_doc.merge(all_kif_accs,left_on = "KIF",right_on = "kif_acc")

In [149]:
kif_jp_uniprot

,Entry,Entry name,Status,Protein names,Gene names,Organism,Organism ID,Taxonomic lineage (all),Sequence,Length
0,Q7PG43,Q7PG43_ANOGA,unreviewed,Kinesin-like protein,1271664 AgaP_AGAP000561,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MSGVREIPAEDSIKVVCRFRPLNDSEELAGSKFVVKFPSGPEENCL...,983
1,Q7PNB7,Q7PNB7_ANOGA,unreviewed,AGAP007815-PA (Fragment),AgaP_AGAP007815,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",LLTNVFSLSLFLGHRFRIRPQIPRELIDMCRVCTQVTPGEPQVLLG...,1033
2,Q7PTK6,Q7PTK6_ANOGA,unreviewed,Kinesin-like protein,1269313 AgaP_AGAP002248,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MDSRIPKPSFLKKPTGPLSLPGNARLPLTRDLLNLPSANSTMFAKV...,762
3,Q7QJN4,Q7QJN4_ANOGA,unreviewed,AGAP007592-PA,AgaP_AGAP007592,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MSDKIRVAVRVRPFNRRELELATENVIEMNGTQTILKYPASLDKME...,1944
4,Q7QDS6,Q7QDS6_ANOGA,unreviewed,Kinesin-like protein (Fragment),AgaP_AGAP010396,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MDRTIKTRSNLSNTKNECVQVVVRCRPLNNKELTGNFQKVVDVFPS...,781
...,...,...,...,...,...,...,...,...,...,...
461,Q93XF8,Q93XF8_MAIZE,unreviewed,Kinesin-like protein (Fragment),KIN6,Zea mays (Maize),4577,"cellular organisms, Eukaryota, Viridiplantae, ...",ELKGNIRVFCRVRPLLRFEGDSNGPEGASISFPTSVESTGRSIDLI...,347
462,Q93XF7,Q93XF7_MAIZE,unreviewed,Kinesin heavy chain (Fragment),KIN8,Zea mays (Maize),4577,"cellular organisms, Eukaryota, Viridiplantae, ...",TRPILPLPDPHPLLSPPPLDADILKSKPPQPARPALLSAVACAMSS...,1079
463,Q93XF5,Q93XF5_MAIZE,unreviewed,Kinesin-like protein (Fragment),KIN11,Zea mays (Maize),4577,"cellular organisms, Eukaryota, Viridiplantae, ...",VQSALDGYKVCIFAYGQTGSGKTYTMMGNPELEDQKGMIPRSLEQI...,354
464,Q93XF3,Q93XF3_MAIZE,unreviewed,Kinesin-like protein (Fragment),KIN15,Zea mays (Maize),4577,"cellular organisms, Eukaryota, Viridiplantae, ...",LRPLSDKEISRGDPAEWECINDTTIISRSTFPDRPTAPTAYSFDRV...,897


In [154]:
more_uniprot = pd.DataFrame()
for acc in (set(acc_ls) - set(kif_jp_uniprot["Entry"])):
    print(acc)
    curr = get_uniprot_entry(acc)
    if not (curr.empty):
        more_uniprot = pd.concat([more_uniprot,curr], ignore_index=True)
    if i%100 ==0:
        print(more_uniprot.shape)
#     break
    

Q2NL05
Q2VIQ3
Q940B8
B7ZNG0
F1M4A4
Q922S8
Q17BU3
P48467
Q10E64
Q5R706
Q29DY1
Q5JKW1
Q2PQA9
Q9BW19
B7EJ91
Q965T6
Q75HV1
Q6P9L6
Q6K765
A6H750
Q498L9
B9EY52
Q9S7P3
Q4KLL9
Q6P9P6
F8WLE0
Q0E2L3
Q5R9K7
B9FMJ3
O35071
B1AVY7
Q1MTQ7
Q5ZLK6
Q75LL2
F1M5N7
Q2M1P5
Q91783
Q86Y91
Q5T7B8
B9G8P1
B9FTR1
Q9QXL1
B7ZC32
Q29RT6
Q0JLE4
Q5W7C6
A3BFT0
L0N7N1
Q6YUL8
Q5ZKV8
B9F2Y7
B9F7C8
Q5XI51
B2GU58
Q6QLM7
Q99PT9
Q28WQ1
A0A068FIK2
F4IBQ9
Q27IK7
B9FL70
P45962
Q2R2P7
Q9P2P6
Q5XI63
F9W301
A2ZRG4
Q4R628
P56536
Q58G59
Q5R9Y9
Q62909
Q5R4H3
Q9WV63
Q6YZ52
B9GE13
B9EUM5
D3YXS5
A0JN40
Q8T135
Q7XPJ0
Q54TL0
Q0WN69
B9FUF9
Q7M6Z5


In [155]:
more_uniprot.shape

(95, 10)

In [158]:
# join the more_uniprot and kif_jp_uniprot table to kif_uniprot
kif_uniprot = pd.concat([kif_jp_uniprot,more_uniprot], ignore_index=True)

In [160]:
kif_uniprot

,Entry,Entry name,Status,Protein names,Gene names,Organism,Organism ID,Taxonomic lineage (all),Sequence,Length
0,Q7PG43,Q7PG43_ANOGA,unreviewed,Kinesin-like protein,1271664 AgaP_AGAP000561,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MSGVREIPAEDSIKVVCRFRPLNDSEELAGSKFVVKFPSGPEENCL...,983
1,Q7PNB7,Q7PNB7_ANOGA,unreviewed,AGAP007815-PA (Fragment),AgaP_AGAP007815,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",LLTNVFSLSLFLGHRFRIRPQIPRELIDMCRVCTQVTPGEPQVLLG...,1033
2,Q7PTK6,Q7PTK6_ANOGA,unreviewed,Kinesin-like protein,1269313 AgaP_AGAP002248,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MDSRIPKPSFLKKPTGPLSLPGNARLPLTRDLLNLPSANSTMFAKV...,762
3,Q7QJN4,Q7QJN4_ANOGA,unreviewed,AGAP007592-PA,AgaP_AGAP007592,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MSDKIRVAVRVRPFNRRELELATENVIEMNGTQTILKYPASLDKME...,1944
4,Q7QDS6,Q7QDS6_ANOGA,unreviewed,Kinesin-like protein (Fragment),AgaP_AGAP010396,Anopheles gambiae (African malaria mosquito),7165,"cellular organisms, Eukaryota, Opisthokonta, M...",MDRTIKTRSNLSNTKNECVQVVVRCRPLNNKELTGNFQKVVDVFPS...,781
...,...,...,...,...,...,...,...,...,...,...
556,Q7XPJ0,KN14I_ORYSJ,reviewed,Kinesin-like protein KIN-14I (Kinesin-like cal...,KIN14I KCBP Os04g0666900 LOC_Os04g57140 OsJ_16...,Oryza sativa subsp. japonica (Rice),39947,"cellular organisms, Eukaryota, Viridiplantae, ...",MNGGGASGGDGYDSDGYSFAPPTPTTLSMSIPPELAGAIPLIDRFQ...,1248
557,Q54TL0,KIF7_DICDI,reviewed,Kinesin-related protein 7 (Kinesin family memb...,kif7 K7 ksnG DDB_G0281555,Dictyostelium discoideum (Slime mold),44689,"cellular organisms, Eukaryota, Amoebozoa, Evos...",MESPVVEGNSGEVATPTLPQPPTPVSSNIRVVCRVRPLTELEKGRN...,1255
558,Q0WN69,KN14P_ARATH,reviewed,Kinesin-like protein KIN-14P,KIN14P At1g73860 F25P22.28 F2P9.27,Arabidopsis thaliana (Mouse-ear cress),3702,"cellular organisms, Eukaryota, Viridiplantae, ...",MNPMRDQPGSPYGDSTPRSPFSPFSPLSVDDRHRNHADTKTPRSPF...,1025
559,B9FUF9,KN12E_ORYSJ,reviewed,Kinesin-like protein KIN-12E,KIN12E Os07g0638000 LOC_Os07g44400 OJ1340_C08....,Oryza sativa subsp. japonica (Rice),39947,"cellular organisms, Eukaryota, Viridiplantae, ...",MAGHGAGGRRASTSRAAARRVEAETNENDDLAAAAARDLAAAEVPA...,1892


In [161]:
# kif uniprot contains 2 set of uniprot sequences entry:
# kif japan table contains protein accession and was used to directly query uniprot
# kif duke: for the entries with documentation on speed and direction, blast against swissprot 
# and get all the accessions corresponding to each entry, then the uniprot accession was used to query uniprot
# sequence and further information

# we could query the sequence by first selecting an entry in kif_doc, then use the entry stored to query sequence
kif_uniprot.to_csv("../../data/kif/kif_uniprot.csv",index = False)

In [162]:
kif_doc

,Species/protein,Molecular mass (kDa),Motor polarity & velocity,Subcellular localization,Comments,Seq,Type,KIF,aa_seq,kif_acc,uniprot_acc
0,D. melanogaster KHC,110,"Plus, 54 µm/min",Diffuse in cytoplasm,Required for viability & neuromuscular function,DmKHC AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEEN...,1,DmKHC,AEDSIKVVCRFRPLNDSEEKAGSKFVVKFPNNVEENCISIA-----...,DmKHC,P17210;P21613;P35978;P28738;O60282;Q61768;Q2PQ...
1,L. pealii KHC,109,"Plus, 30 µm/min",Membranous vesicles,NaN,LpKHC SECNIKVICRVRPLNEAEERAGSKFILKFP---TDD...,1,LpKHC,SECNIKVICRVRPLNEAEERAGSKFILKFP---TDDSISIA-----...,LpKHC,P21613;P17210;O60282;P28738;P33175;Q12840;P359...
2,N. crassa KHC (NKin),103,"Plus, 120-180 µm/min",ND,High velocity of microtubule transport; no cop...,NcKHC SANSIKVVARFRPQNRVEIESGGQPIVTFQ---GPD...,1,NcKHC,SANSIKVVARFRPQNRVEIESGGQPIVTFQ---GPDTCTVD---SK...,NcKHC,P48467;Q86Z98;Q86ZC1;O43093;Q9US60;Q54UC9;P331...
3,D. melanogaster Klp68D,88,"Plus, 18 µm/min",ND,NaN,Dmklp68D PNECVQVVVRCRPMSNRERSERSPEVVNVYPNRGVV...,2,Dmklp68D,PNECVQVVVRCRPMSNRERSERSPEVVNVYPNRGVVELQNVV----...,Dmklp68D,
4,M. musculus KIF3A,80,"Plus, 36 µm/min",Microsomes/ synaptic vesicles,NaN,MmKIF3A SCDNVKVVVRCRPLNEREKSMCYRQAVSVDEMRGTI...,2,MmKIF3A,SCDNVKVVVRCRPLNEREKSMCYRQAVSVDEMRGTITVHKTD----...,MmKIF3A,P28741;Q4R628;Q9Y496;Q5R4H3;P46872;P46871;Q617...
5,M. musculus KIF3B,85,"Plus, 18 µm/min","Neurons, cell bodies, axons and dendrites",NaN,MmKIF3B SSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQV...,2,MmKIF3B,SSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQVSVKNPK----...,MmKIF3B,Q61771;O15066;P46871;Q5R706;O35066;O55165;O147...
6,S. purpuratus KRP85,79,"Plus, 24 µm/min",ND,NaN,SpKRP85 GNDNVRVVVRCRPLNSKETGQGFKSVVKMDEMRGTV...,2,SpKRP85,GNDNVRVVVRCRPLNSKETGQGFKSVVKMDEMRGTVQVTNPN----...,SpKRP85,P46872;Q4R628;Q9Y496;Q5R4H3;P28741;P46871;Q617...
7,S. purpuratus KRP95,84,"Plus, 24 µm/min",ND,NaN,SpKRP95 SAETVKVVVRCRPMNSKEISQGHKRIVEMDNKRGLV...,2,SpKRP95,SAETVKVVVRCRPMNSKEISQGHKRIVEMDNKRGLVEVTNPK----...,SpKRP95,P46871;O15066;Q61771;P46872;Q5R706;O14782;O350...
8,M. musculus KIF1A,192,"Plus, 72 µm/min",Enriched in axons,"Associated with synaptic vesicles, monomeric?",MmKIF1A AGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTI...,3,MmKIF1A,AGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPK-----Q...,MmKIF1A,P33173;F1M4A4;Q12756;O60333;Q60575;O88658;O438...
9,M. musculus KIF1B,130,"Plus, 40 µm/min",Co-localizes with mitochondria,"Transports mitochondria, monomeric?",MmKIF1B SGASVKVAVRVRPFNSRETSKESKCIIQMQGNSTSI...,3,MmKIF1B,SGASVKVAVRVRPFNSRETSKESKCIIQMQGNSTSIINPK-----N...,MmKIF1B,Q60575;O88658;O60333;Q12756;F1M4A4;P33173;O350...


In [163]:
kif_doc.to_csv("../../data/kif/kif_speed.csv",index = False)